In [1]:
import pandas as pd
from collections import Counter
from tqdm import tqdm
import ast
from gensim.models import FastText

In [ ]:
# df = pd.read_csv('cleaned_data_keywords.txt')

In [ ]:
# df.columns

Index(['date', 'sentences'], dtype='object')

### Fast-text는 txt파일이어야함

In [ ]:
# 문장 변환
import pandas as pd
import ast

# 데이터 불러오기
df = pd.read_csv('hk_sentences_1.csv')

# 'sentences' 칼럼에서 태깅을 제거하고 단어만 추출 (태그 분리)
df['text'] = df['sentences'].apply(lambda x: ' '.join([word.split('/')[0] for word in ast.literal_eval(x)]))

# 텍스트 파일로 저장
with open('hk_sentences.txt', 'w', encoding='utf-8') as f:
    for text in df['text']:
        f.write(text + '\n')



In [ ]:
import pandas as pd
import ast

df = pd.read_csv('hk_sentences_1.csv')

# 태깅 제거
df['text'] = df['sentences'].apply(lambda x: ' '.join([word[0] for word in ast.literal_eval(x)]))

# 텍스트 파일로 따로 저장해보자
with open('hk_sentences.txt', 'w', encoding='utf-8') as f:
    for text in df['text']:
        f.write(text + '\n')


### n그램 내장함수? 편하다

In [ ]:
# text 파일 읽기
with open('/content/sample_data/cleaned_data_keywords.txt', 'r', encoding='utf-8') as f:
    tokens = [line.strip().split() for line in f.readlines()] #리스트

In [ ]:
from gensim.models import FastText

# 학습
model = FastText(
    sentences=tokens,  # tokens는 텍스트 파일에서 읽은 단어 리스트여야 함
    vector_size=300, 
    window=5,
    min_count=25,
    sg=1,
    epochs=3,
    min_n=1,
    max_n=5,
    negative=5
)

# 모델 저장
model.save("업데이트.bin")


#### 순도 높은 사전을 얻지 못한 것 같다. 
    -(논문 그대로)
    vector_size=128 > 300으로 늘려버리기 > 메모리 과부하
    min_n=1 ~ max_n=5 (n그램)
    negative= 5

In [2]:
# 모델 로드
model = FastText.load("업데이트.bin")

# 예시: 모델 사용
print(model.wv["금리인상"])  # 특정 단어 입력, FastText 모델은 복합단어 가능


[ 0.28488445 -0.3065864   0.0548155  -0.00822149  0.07313827  0.15408309
 -0.14688192  0.13730937  0.03785319  0.01474491 -0.15732452  0.22435066
 -0.07345411  0.27354822 -0.10147103  0.02262285 -0.00062848 -0.12988651
 -0.1522029  -0.0818905   0.18118075  0.2066023   0.02247971 -0.08116864
 -0.22655866  0.16399886 -0.13758014 -0.34404016  0.39889205  0.13880464
  0.4040078  -0.01741497  0.31090698 -0.03572432 -0.14591941  0.07354025
  0.04738261 -0.09606393 -0.03362769  0.20676804 -0.21234478 -0.08590214
 -0.07912035 -0.0200071   0.22323547  0.1599033  -0.10702426  0.23773159
 -0.01360496  0.19907564  0.09832553 -0.12158774 -0.02985627  0.14041013
 -0.16591775 -0.01259443 -0.04550545 -0.11344734  0.01583545  0.01595455
 -0.04837182 -0.25290856 -0.10143162 -0.05382365 -0.10247338 -0.06948587
 -0.18525155  0.2576035  -0.11511842  0.30417836  0.03564642 -0.21197245
  0.14050514 -0.02361164 -0.05550092 -0.20660576  0.1909718  -0.0184369
 -0.11083558 -0.00177996 -0.10392036  0.30232647 -0.

### 벡터

In [ ]:
from gensim.models import FastText, Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

#모델 부르기
fasttext_model = FastText.load("업데이트.bin")


# 예시 단어 벡터 추출
fasttext_vector_1 = fasttext_model.wv['금리인하']
fasttext_vector_2 = fasttext_model.wv['매파']


# 유사도 계산 (Cosine Similarity)
cosine_sim_fasttext = cosine_similarity([fasttext_vector_1], [fasttext_vector_2])[0][0]

# 출력
print(f"FastText 모델에서 '금리인상'과 '매파'의 코사인 유사도: {cosine_sim_fasttext}")



FastText 모델에서 '금리인상'과 '매파'의 코사인 유사도: 0.43686261773109436


In [ ]:
from gensim.models import FastText, Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
fasttext_model = FastText.load("전체.bin")
fasttext_vector_apple = fasttext_model.wv['인상']
fasttext_vector_banana = fasttext_model.wv['비둘기파']
cosine_sim_fasttext = cosine_similarity([fasttext_vector_apple], [fasttext_vector_banana])[0][0]
print('인상' in fasttext_model.wv.key_to_index)
print('비둘기파' in fasttext_model.wv.key_to_index)
print(f"FastText 모델에서 '금리인상'과 '매파'의 코사인 유사도: {cosine_sim_fasttext}")



True
True
FastText 모델에서 '금리인상'과 '매파'의 코사인 유사도: 0.5579149723052979


In [12]:
from gensim.models import FastText

model_path = "업데이트.bin"  # 정확한 파일 경로 입력
model = FastText.load(model_path)  # 모델 불러오기
import numpy as np
from scipy.spatial.distance import cosine

# 감성 단어 리스트 (예제)
hawkish_words = [
    "강경", "공격적", "제재", "압박", "보복", "군사", "대응", "대립", "분쟁", "전쟁", "위험", "자극",
    "불안", "위기", "공격", "반격", "저항", "독재", "억제", "억압", "단호", "무력", "충돌", "전술",
    "철강", "경제적 압박", "무역전쟁", "대립각", "강경책", "복수", "강력한", "신속한", "도발", "세력", "공격적인"
]

dovish_words = [
    "평화", "협력", "완화", "타협", "외교", "대화", "조정", "중재", "합의", "화해", "안정", "협상",
    "협약", "교류", "관계", "회담", "평화적", "소통", "융화", "유화", "국제적", "상호이해", "신뢰",
    "공동체", "존중", "연대", "공존", "구호", "공동선", "평화주의", "유연한", "지속적", "소극적", "융통성",
    "친선", "구속력", "구속적"
]

# 단어 임베딩 벡터 평균 구하기
def get_average_vector(words, model):
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else None

hawkish_vector = get_average_vector(hawkish_words, model)
dovish_vector = get_average_vector(dovish_words, model)

# 감성 점수 계산 함수
def get_polarity_score(word, model, hawkish_vector, dovish_vector):
    if word not in model.wv:
        return None

    word_vector = model.wv[word]
    hawkish_sim = 1 - cosine(word_vector, hawkish_vector)
    dovish_sim = 1 - cosine(word_vector, dovish_vector)

    return hawkish_sim / dovish_sim  # 비율 기반 극성 점수

# 테스트 실행
test_word = "비둘기파"
print(get_polarity_score(test_word, model, hawkish_vector, dovish_vector))



1.0980782008403298


### 부트스트래핑 반복

In [ ]:
import numpy as np
from scipy.spatial.distance import cosine

# 감성 단어 리스트
hawkish_words = [
    "강경", "공격적", "제재", "압박", "보복", "군사", "대응", "대립", "분쟁", "전쟁", "위험", "자극",
    "불안", "위기", "공격", "반격", "저항", "독재", "억제", "억압", "단호", "무력", "충돌", "전술",
    "철강", "경제적 압박", "무역전쟁", "대립각", "강경책", "복수", "강력한", "신속한", "도발", "세력", "공격적인"
]

dovish_words = [
    "평화", "협력", "완화", "타협", "외교", "대화", "조정", "중재", "합의", "화해", "안정", "협상",
    "협약", "교류", "관계", "회담", "평화적", "소통", "융화", "유화", "국제적", "상호이해", "신뢰",
    "공동체", "존중", "연대", "공존", "구호", "공동선", "평화주의", "유연한", "지속적", "소극적", "융통성",
    "친선", "구속력", "구속적"
]

# 감성 단어를 10개의 하위 집합으로 분할
hawkish_subsets = np.array_split(hawkish_words, 10)
dovish_subsets = np.array_split(dovish_words, 10)

# 부트스트래핑(랜덤 샘플링) 50회 반복
boot_polarity_scores = []

for _ in range(1000):
    # 랜덤 하위 집합 선택
    hawkish_sample = np.random.choice(hawkish_words, size=len(hawkish_words)//2, replace=False)
    dovish_sample = np.random.choice(dovish_words, size=len(dovish_words)//2, replace=False)

    # 감성 벡터 업데이트
    hawkish_vector = get_average_vector(hawkish_sample, model)
    dovish_vector = get_average_vector(dovish_sample, model)

    # 특정 단어 감성 점수 계산 
    test_word = "인하"
    score = get_polarity_score(test_word, model, hawkish_vector, dovish_vector)

    if score is not None:
        boot_polarity_scores.append(score)

# 평균 감성 점수 출력
final_polarity_score = np.mean(boot_polarity_scores)
print(f"부트스트래핑 평균 극성 점수: {final_polarity_score:.3f}")


부트스트래핑 평균 극성 점수: 0.987


### 말도 안되는 점수에 재수정

In [20]:
import random
from gensim.models import FastText
import numpy as np

model = FastText.load("업데이트.bin")
hawkish_words = [
    "강경", "공격적", "제재", "압박", "보복", "군사", "대응", "대립", "분쟁", "전쟁",
    "위험", "자극", "불안", "위기", "공격", "반격", "저항", "독재", "억제", "억압",
    "단호", "무력", "충돌", "전술", "긴축", "인상", "상승", "팽창", "성장", "투기",
    "상회", "광려", "확장", "상방", "흑자", "견조", "낙관", "매파", "변동성 감소",
    "금리 상승", "물가 상승", "인플레이션 압력", "위험 선호", "채권 가격 하락", "요금 인상",
    "부동산 가격 상승"
]

dovish_words = [
    "평화", "협력", "완화", "타협", "외교", "대화", "조정", "중재", "합의", "화해",
    "안정", "협상", "협약", "교류", "관계", "회담", "소통", "융화", "유화", "신뢰",
    "공존", "존중", "연대", "인하", "둔화", "하락", "감소", "위축", "침체", "하회",
    "비둘기", "하방", "적자", "부진", "비관", "변동성 확대", "금리 하락", "물가 하락",
    "위험 회피", "채권 가격 상승", "요금 인하", "부동산 가격 하락"
]

def bootstrap_lexicon(seed_words, model, iterations=50, subset_size=10):
    expanded_lexicon = set(seed_words)
    for _ in range(iterations):
        subset = random.sample(seed_words, min(subset_size, len(seed_words)))
        subset_vector = get_average_vector(subset, model)
        similar_words = model.wv.most_similar(positive=[subset_vector], topn=10)
        expanded_lexicon.update([word for word, _ in similar_words])
    return expanded_lexicon

hawkish_expanded = bootstrap_lexicon(hawkish_words, model)
dovish_expanded = bootstrap_lexicon(dovish_words, model)

### 매파 비둘기파 나눠버리기

In [ ]:
def build_polarity_lexicon(model, hawkish_seeds, dovish_seeds):
    hawkish_lexicon = []
    dovish_lexicon = []
    for word in model.wv.index_to_key:
        score = sentiment_propagation(word, model, hawkish_seeds, dovish_seeds)
        if score > 1.1:
            hawkish_lexicon.append(word)
        elif score < 0.9:
            dovish_lexicon.append(word)
    return hawkish_lexicon, dovish_lexicon

hawkish_lexicon, dovish_lexicon = build_polarity_lexicon(model, hawkish_words, dovish_words)
print(f"Hawkish words: {len(hawkish_lexicon)}, Dovish words: {len(dovish_lexicon)}")